In [1]:
import pandas as pd
import numpy as np

In [2]:
contrats = pd.read_excel('../data/Données_Assurance_S2.xlsx',sheet_name='Contrats')

In [3]:
contrats.head()

,ContratID,ClientID,Branche,Produit,DateEffet,DateEcheance,StatutContrat,StatutPaiement,PrimeAnnuelle
0,1,6915,INCENDIE,INCENDIE AUTRES,2023-09-17,2027-05-18,Suspendu,Non payé,3598
1,2,6102,ENGINEERING,RESPONSABILITE DECENNALE,2022-03-16,2024-09-18,Actif,À jour,1319
2,3,1610,INCENDIE,INCENDIE RISQUES AGRICOLES,2024-03-06,2026-06-04,Résilié,À jour,1498
3,4,4828,VIE,ASSURANCE VIE COMPLEMENT RETRAITE - HORIZON+ (AS),2021-06-27,2025-03-19,Suspendu,À jour,3255
4,5,1950,RISQUES DIVERS,R.C & INDIVIDUELLE COMPLEMENTAIRE PECHE ET PL...,2021-04-19,2026-12-05,Résilié,À jour,5332


In [4]:
clients = pd.read_pickle('../data/clients.pkl')

In [5]:
clients.head()

,ClientID,Nom,Prénom,CIN,DateNaissance,Sexe,SituationFamiliale,Profession,RevenusMensuels,Adresse,Email,Téléphone,WhatsApp,age
0,1,Ben Ali El Amri,Ahmed Salah,82387512,1981-12-07,F,Marié,Médecin,4041,"97 Rue Imbert - Mutuelleville, Sfax",ahmed.salah@gmail.com,21671364363,21673804239,44
1,2,Ben Ali Ben Hassen,Ahmed Salah,11727315,1998-02-28,M,Marié,Artisan,1813,"106 Rue Giraud - Centre-Ville, Mahdia",ahmed.salah@gmail.com,21623740587,21694324932,27
2,3,Ben Ali Tlili,Ahmed Salah,82121072,1980-04-13,M,Marié,Médecin,2867,"164 Rue Guichard - Lac 2, Gafsa",ahmed.salah@gmail.com,21667576259,21689757319,45
3,4,Ben Ali Belhaj,Ahmed Salah,2858647,2006-01-13,M,Marié,Ingénieur,2994,"156 Rue Colas - Centre-Ville, Sfax",ahmed.salah@gmail.com,21638268772,21694185357,19
4,5,Ben Ali Mzoughi,Ahmed Salah,52452825,1963-01-15,F,Divorcé,Étudiant,5729,"46 Rue Weber - Ezzahra, Kasserine",ahmed.salah@gmail.com,21679949500,21691349318,62


Merging clients and contrats

In [6]:
merged = pd.merge(
    clients,
    contrats,
    on='ClientID',
    how='inner'
)

merged.head()

In [7]:
merged = merged.drop(columns=['DateNaissance'])
merged.head()

,ClientID,Nom,Prénom,CIN,Sexe,SituationFamiliale,Profession,RevenusMensuels,Adresse,Email,...,WhatsApp,age,ContratID,Branche,Produit,DateEffet,DateEcheance,StatutContrat,StatutPaiement,PrimeAnnuelle
0,1,Ben Ali El Amri,Ahmed Salah,82387512,F,Marié,Médecin,4041,"97 Rue Imbert - Mutuelleville, Sfax",ahmed.salah@gmail.com,...,21673804239,44,8893,VIE,RENTE EDUCATION (AS),2023-04-29,2025-05-07,Actif,Retard,3092
1,1,Ben Ali El Amri,Ahmed Salah,82387512,F,Marié,Médecin,4041,"97 Rue Imbert - Mutuelleville, Sfax",ahmed.salah@gmail.com,...,21673804239,44,25539,VIE,CREDITS COMMERCIAUX (AS),2024-06-19,2025-05-23,Actif,À jour,2057
2,1,Ben Ali El Amri,Ahmed Salah,82387512,F,Marié,Médecin,4041,"97 Rue Imbert - Mutuelleville, Sfax",ahmed.salah@gmail.com,...,21673804239,44,53883,TRANSPORT,POLICE AU VOYAGE(FACULTE MARITIME),2024-02-28,2024-11-06,Suspendu,Retard,3632
3,1,Ben Ali El Amri,Ahmed Salah,82387512,F,Marié,Médecin,4041,"97 Rue Imbert - Mutuelleville, Sfax",ahmed.salah@gmail.com,...,21673804239,44,73740,TRANSPORT,POLICE A ALIMENTER(CORPS DE PECHE),2023-08-16,2027-05-06,Résilié,Non payé,5101
4,2,Ben Ali Ben Hassen,Ahmed Salah,11727315,M,Marié,Artisan,1813,"106 Rue Giraud - Centre-Ville, Mahdia",ahmed.salah@gmail.com,...,21694324932,27,13369,AUTOMOBILE,AUTOMOBILE ASSURANCE FRONTIERE TEMPORAIRE,2021-08-20,2026-01-26,Suspendu,À jour,751


In [8]:
merged.shape

(80000, 21)

Client Scoring

In [9]:
def calculate_client_score(group):
    contrat_score = 0
    payment_score = 0
    premium_score = 0
    
    contrat_weight = 0.4
    payment_weight = 0.4
    premium_weight = 0.2
    
    contrat_counts = group['StatutContrat'].value_counts(normalize=True)
    contrat_score = (
        (contrat_counts.get('Actif',0)*1.0)+
        (contrat_counts.get('Suspendu',0)*0.3
         + contrat_counts.get('Résilié',0) * -0.5
        )
    )
    payment_counts = group['StatutPaiement'].value_counts(normalize=True)
    payment_score = (
        (payment_counts.get('A jour', 0) * 1.0) + 
        (payment_counts.get('Retard', 0) * -0.5) + 
        (payment_counts.get('Non Payé', 0) * -1.0
    ))
    if group['PrimeAnnuelle'].sum() > 0:
        premium_score = np.log1p(group['PrimeAnnuelle'].sum())  
        composite_score = (
        (contrat_score * contrat_weight) + 
        (payment_score * payment_weight) + 
        (premium_score * premium_weight)
    )
    return composite_score

In [10]:
clients_scores = merged.groupby('ClientID').apply(calculate_client_score).reset_index()
clients_scores.columns = ['ClientID','ClientScore']

C:\Users\ghass\AppData\Local\Temp\ipykernel_2096\2839576852.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  clients_scores = merged.groupby('ClientID').apply(calculate_client_score).reset_index()


In [11]:
clients_scores.head()

,ClientID,ClientScore
0,1,1.987684
1,2,1.937188
2,3,2.015942
3,4,2.113921
4,5,1.916701


In [12]:
merged = merged.merge(clients_scores, on='ClientID', how='left')
merged['ClientScore'] = (merged['ClientScore'] - merged['ClientScore'].min()) / (merged['ClientScore'].max() - merged['ClientScore'].min()) * 100

In [13]:
merged.head()

,ClientID,Nom,Prénom,CIN,Sexe,SituationFamiliale,Profession,RevenusMensuels,Adresse,Email,...,age,ContratID,Branche,Produit,DateEffet,DateEcheance,StatutContrat,StatutPaiement,PrimeAnnuelle,ClientScore
0,1,Ben Ali El Amri,Ahmed Salah,82387512,F,Marié,Médecin,4041,"97 Rue Imbert - Mutuelleville, Sfax",ahmed.salah@gmail.com,...,44,8893,VIE,RENTE EDUCATION (AS),2023-04-29,2025-05-07,Actif,Retard,3092,75.203415
1,1,Ben Ali El Amri,Ahmed Salah,82387512,F,Marié,Médecin,4041,"97 Rue Imbert - Mutuelleville, Sfax",ahmed.salah@gmail.com,...,44,25539,VIE,CREDITS COMMERCIAUX (AS),2024-06-19,2025-05-23,Actif,À jour,2057,75.203415
2,1,Ben Ali El Amri,Ahmed Salah,82387512,F,Marié,Médecin,4041,"97 Rue Imbert - Mutuelleville, Sfax",ahmed.salah@gmail.com,...,44,53883,TRANSPORT,POLICE AU VOYAGE(FACULTE MARITIME),2024-02-28,2024-11-06,Suspendu,Retard,3632,75.203415
3,1,Ben Ali El Amri,Ahmed Salah,82387512,F,Marié,Médecin,4041,"97 Rue Imbert - Mutuelleville, Sfax",ahmed.salah@gmail.com,...,44,73740,TRANSPORT,POLICE A ALIMENTER(CORPS DE PECHE),2023-08-16,2027-05-06,Résilié,Non payé,5101,75.203415
4,2,Ben Ali Ben Hassen,Ahmed Salah,11727315,M,Marié,Artisan,1813,"106 Rue Giraud - Centre-Ville, Mahdia",ahmed.salah@gmail.com,...,27,13369,AUTOMOBILE,AUTOMOBILE ASSURANCE FRONTIERE TEMPORAIRE,2021-08-20,2026-01-26,Suspendu,À jour,751,72.129215


In [14]:
merged.shape

(80000, 22)

In [15]:
merged.columns

Index(['ClientID', 'Nom', 'Prénom', 'CIN', 'Sexe', 'SituationFamiliale',
       'Profession', 'RevenusMensuels', 'Adresse', 'Email', 'Téléphone',
       'WhatsApp', 'age', 'ContratID', 'Branche', 'Produit', 'DateEffet',
       'DateEcheance', 'StatutContrat', 'StatutPaiement', 'PrimeAnnuelle',
       'ClientScore'],
      dtype='object')

In [16]:
client_data = merged.groupby('ClientID').agg({
    'Nom':'first',
    'Prénom':'first',
    'CIN':'first',
    'Sexe':'first',
    'SituationFamiliale':'first',
    'Profession':'first',
    'RevenusMensuels':'first',
    'Adresse':'first',
    'Email':'first',
    'Téléphone':'first',
    'WhatsApp':'first',
    'age':'first',
    'ContratID':'first',
    'Branche':'first',
    'Produit':list,
    'DateEffet':lambda x:x.value_counts().to_dict(),
    'DateEcheance':lambda x:x.value_counts().to_dict(),
    'StatutContrat':lambda x:x.value_counts().to_dict(),
    'StatutPaiement':lambda x:x.value_counts().to_dict(),
    'PrimeAnnuelle': 'sum'
}).reset_index()

In [17]:
client_data = client_data.merge(clients_scores,on='ClientID',how='left')

In [18]:
client_data.shape

(14343, 22)

In [19]:
client_data.head()

,ClientID,Nom,Prénom,CIN,Sexe,SituationFamiliale,Profession,RevenusMensuels,Adresse,Email,...,age,ContratID,Branche,Produit,DateEffet,DateEcheance,StatutContrat,StatutPaiement,PrimeAnnuelle,ClientScore
0,1,Ben Ali El Amri,Ahmed Salah,82387512,F,Marié,Médecin,4041,"97 Rue Imbert - Mutuelleville, Sfax",ahmed.salah@gmail.com,...,44,8893,VIE,"[RENTE EDUCATION (AS), CREDITS COMMERCIAUX (AS...","{2023-04-29 00:00:00: 1, 2024-06-19 00:00:00: ...","{2025-05-07 00:00:00: 1, 2025-05-23 00:00:00: ...","{'Actif': 2, 'Suspendu': 1, 'Résilié': 1}","{'Retard': 2, 'À jour': 1, 'Non payé': 1}",13882,1.987684
1,2,Ben Ali Ben Hassen,Ahmed Salah,11727315,M,Marié,Artisan,1813,"106 Rue Giraud - Centre-Ville, Mahdia",ahmed.salah@gmail.com,...,27,13369,AUTOMOBILE,"[AUTOMOBILE ASSURANCE FRONTIERE TEMPORAIRE, DE...","{2021-08-20 00:00:00: 1, 2023-04-05 00:00:00: ...","{2026-01-26 00:00:00: 1, 2027-01-17 00:00:00: ...","{'Résilié': 3, 'Actif': 2, 'Suspendu': 1}","{'À jour': 2, 'Retard': 2, 'Non payé': 2}",17198,1.937188
2,3,Ben Ali Tlili,Ahmed Salah,82121072,M,Marié,Médecin,2867,"164 Rue Guichard - Lac 2, Gafsa",ahmed.salah@gmail.com,...,45,1669,GROUPE-MALADIE,"[SANTE ET PREVOYANCE, TOUS RISQUES MONTAGE, OM...","{2021-03-10 00:00:00: 1, 2021-11-30 00:00:00: ...","{2025-08-26 00:00:00: 1, 2027-03-12 00:00:00: ...","{'Actif': 4, 'Résilié': 3, 'Suspendu': 1}","{'Non payé': 4, 'À jour': 2, 'Retard': 2}",15209,2.015942
3,4,Ben Ali Belhaj,Ahmed Salah,2858647,M,Marié,Ingénieur,2994,"156 Rue Colas - Centre-Ville, Sfax",ahmed.salah@gmail.com,...,19,648,GROUPE-MALADIE,"[SANTE ET PREVOYANCE, POLICE A ALIMENTER(CORPS...","{2022-03-18 00:00:00: 1, 2023-08-02 00:00:00: ...","{2025-04-26 00:00:00: 1, 2025-11-27 00:00:00: ...","{'Actif': 4, 'Suspendu': 3, 'Résilié': 1}","{'À jour': 4, 'Retard': 4}",21366,2.113921
4,5,Ben Ali Mzoughi,Ahmed Salah,52452825,F,Divorcé,Étudiant,5729,"46 Rue Weber - Ezzahra, Kasserine",ahmed.salah@gmail.com,...,62,4482,RISQUES DIVERS,[R.C CHEF DE FAMILLE DETENTEUR D UN ANIMAL DOM...,"{2020-12-11 00:00:00: 1, 2022-05-22 00:00:00: ...","{2025-07-12 00:00:00: 1, 2027-04-28 00:00:00: ...","{'Résilié': 1, 'Actif': 1, 'Suspendu': 1}",{'À jour': 3},8519,1.916701


In [20]:
client_data['ClientScore'].agg(['min','max'])

min    0.752410
max    2.394987
Name: ClientScore, dtype: float64

Scaling the client score

In [21]:
client_data['ClientScore'] = (client_data['ClientScore'] - client_data['ClientScore'].min()) / (client_data['ClientScore'].max() - client_data['ClientScore'].min())

In [22]:
client_data['ClientScore'].agg(['min','max'])

min    0.0
max    1.0
Name: ClientScore, dtype: float64

In [23]:
client_data[client_data['ClientScore'] >= 0.7]

,ClientID,Nom,Prénom,CIN,Sexe,SituationFamiliale,Profession,RevenusMensuels,Adresse,Email,...,age,ContratID,Branche,Produit,DateEffet,DateEcheance,StatutContrat,StatutPaiement,PrimeAnnuelle,ClientScore
0,1,Ben Ali El Amri,Ahmed Salah,82387512,F,Marié,Médecin,4041,"97 Rue Imbert - Mutuelleville, Sfax",ahmed.salah@gmail.com,...,44,8893,VIE,"[RENTE EDUCATION (AS), CREDITS COMMERCIAUX (AS...","{2023-04-29 00:00:00: 1, 2024-06-19 00:00:00: ...","{2025-05-07 00:00:00: 1, 2025-05-23 00:00:00: ...","{'Actif': 2, 'Suspendu': 1, 'Résilié': 1}","{'Retard': 2, 'À jour': 1, 'Non payé': 1}",13882,0.752034
1,2,Ben Ali Ben Hassen,Ahmed Salah,11727315,M,Marié,Artisan,1813,"106 Rue Giraud - Centre-Ville, Mahdia",ahmed.salah@gmail.com,...,27,13369,AUTOMOBILE,"[AUTOMOBILE ASSURANCE FRONTIERE TEMPORAIRE, DE...","{2021-08-20 00:00:00: 1, 2023-04-05 00:00:00: ...","{2026-01-26 00:00:00: 1, 2027-01-17 00:00:00: ...","{'Résilié': 3, 'Actif': 2, 'Suspendu': 1}","{'À jour': 2, 'Retard': 2, 'Non payé': 2}",17198,0.721292
2,3,Ben Ali Tlili,Ahmed Salah,82121072,M,Marié,Médecin,2867,"164 Rue Guichard - Lac 2, Gafsa",ahmed.salah@gmail.com,...,45,1669,GROUPE-MALADIE,"[SANTE ET PREVOYANCE, TOUS RISQUES MONTAGE, OM...","{2021-03-10 00:00:00: 1, 2021-11-30 00:00:00: ...","{2025-08-26 00:00:00: 1, 2027-03-12 00:00:00: ...","{'Actif': 4, 'Résilié': 3, 'Suspendu': 1}","{'Non payé': 4, 'À jour': 2, 'Retard': 2}",15209,0.769237
3,4,Ben Ali Belhaj,Ahmed Salah,2858647,M,Marié,Ingénieur,2994,"156 Rue Colas - Centre-Ville, Sfax",ahmed.salah@gmail.com,...,19,648,GROUPE-MALADIE,"[SANTE ET PREVOYANCE, POLICE A ALIMENTER(CORPS...","{2022-03-18 00:00:00: 1, 2023-08-02 00:00:00: ...","{2025-04-26 00:00:00: 1, 2025-11-27 00:00:00: ...","{'Actif': 4, 'Suspendu': 3, 'Résilié': 1}","{'À jour': 4, 'Retard': 4}",21366,0.828887
4,5,Ben Ali Mzoughi,Ahmed Salah,52452825,F,Divorcé,Étudiant,5729,"46 Rue Weber - Ezzahra, Kasserine",ahmed.salah@gmail.com,...,62,4482,RISQUES DIVERS,[R.C CHEF DE FAMILLE DETENTEUR D UN ANIMAL DOM...,"{2020-12-11 00:00:00: 1, 2022-05-22 00:00:00: ...","{2025-07-12 00:00:00: 1, 2027-04-28 00:00:00: ...","{'Résilié': 1, 'Actif': 1, 'Suspendu': 1}",{'À jour': 3},8519,0.708820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14337,14395,Gharbi Mzoughi,Ons Brahim,86245723,F,Célibataire,Artisan,2705,"98 Rue Lamy - La Marsa, Kasserine",ons.brahim@gmail.com,...,30,14507,GROUPE-MALADIE,"[SANTE ET PREVOYANCE, BRIS DE MACHINES, INCEND...","{2020-09-28 00:00:00: 1, 2024-05-10 00:00:00: ...","{2025-12-12 00:00:00: 1, 2026-01-21 00:00:00: ...","{'Actif': 5, 'Résilié': 3}","{'Retard': 3, 'Non payé': 3, 'À jour': 2}",16129,0.782476
14338,14396,Gharbi Cherif,Ons Brahim,16110737,F,Célibataire,Artisan,2001,"117 Rue Delahaye - La Marsa, Sfax",ons.brahim@gmail.com,...,27,6564,TRANSPORT,"[POLICE ABONNEMENT(FACULTE AERIEN), PACK TOUS ...","{2021-11-26 00:00:00: 1, 2021-06-04 00:00:00: ...","{2025-06-13 00:00:00: 1, 2027-05-20 00:00:00: ...","{'Suspendu': 2, 'Résilié': 1, 'Actif': 1}","{'Non payé': 2, 'À jour': 1, 'Retard': 1}",20044,0.784582
14340,14398,Gharbi Mrad,Ons Brahim,26132257,F,Divorcé,Médecin,2788,"49 Rue Guyon - Mutuelleville, Sfax",ons.brahim@gmail.com,...,24,7937,TRANSPORT,[POLICE AU VOYAGE(CORPS DE CONSTRUCTION NAVAL)...,"{2020-10-14 00:00:00: 1, 2022-08-07 00:00:00: ...","{2026-10-29 00:00:00: 1, 2027-03-03 00:00:00: ...","{'Actif': 3, 'Résilié': 3, 'Suspendu': 2}","{'Retard': 4, 'À jour': 3, 'Non payé': 1}",31108,0.804614
14341,14399,Gharbi Dhieb,Ons Brahim,37229641,M,Célibataire,Médecin,5484,"58 Rue Rousset - Lac 2, Gafsa",ons.brahim@gmail.com,...,51,12996,ENGINEERING,"[TOUS RISQUES CHANTIER, DHAMEN SFDGUI, CREDITS...","{2021-03-09 00:00:00: 1, 2024-06-10 00:00:00: ...","{2025-10-03 00:00:00: 1, 2026-07-28 00:00:00: ...","{'Résilié': 2, 'Actif': 1, 'Suspendu': 1}","{'À jour': 2, 'Non payé': 1, 'Retard': 1}",16440,0.711745


In [24]:
def recommander_assurances(age, situation_familiale, profession, revenus_mensuels):
    recommendations = []

    # 1. Assurance VIE (Épargne long terme)
    if (25 <= age <= 60) and (revenus_mensuels >= 3000) and (situation_familiale in ["Marié", "Divorcé"]):
        recommendations.append("AHMINI ASSURANCE")

    # 2. Assurance RISQUES DIVERS (RC Professionnelle)
    if profession == 'Médecin' and revenus_mensuels >= 2000:
        recommendations.append('R.C MEDECIN')
    if profession == 'Artisan' and revenus_mensuels >= 2000:
        recommendations.append('RC ARTISANTS ET COMMERCANTS')

    # 3. Assurance AUTOMOBILE
    if (18 <= age <= 70) and revenus_mensuels >= 1500:
        if profession not in ["Étudiant", "Retraité"]:  # Filtre métiers peu susceptibles d'avoir une voiture
            if age <= 25:
                recommendations.append("AUTOMOBILE AMICALE")
            else:
                recommendations.append("AUTOMOBILE")

    # 4. Assurance TRANSPORT (Marchandises)
    if profession == "Commerçant" and revenus_mensuels >= 4000:
        recommendations.append("POLICE AU VOYAGE(FACULTE TERRESTRE)")

    # 5. Assurance ENGINEERING
    if profession == "Ingénieur" and revenus_mensuels >= 4500:
        recommendations.append("TOUS RISQUES CHANTIER")

    # 6. Assurance INCENDIE
    if revenus_mensuels >= 2500 and profession not in ["Étudiant"]:
        recommendations.append("INCENDIE RISQUES INDUSTRIEL")

    # 7. Assurance GROUPE-MALADIE
    if profession in ["Médecin", "Professeur", "Employé"] and revenus_mensuels >= 3500:
        recommendations.append("SANTE ET PREVOYANCE")

    return recommendations

In [25]:
client_data['recommendations'] = client_data.apply(
    lambda row: recommander_assurances(
        age=row['age'],
        situation_familiale=row['SituationFamiliale'],
        profession=row['Profession'],
        revenus_mensuels=row['RevenusMensuels']
    ),
    axis=1
)

client_data[['Produit','recommendations']].head()

,Produit,recommendations
0,"[RENTE EDUCATION (AS), CREDITS COMMERCIAUX (AS...","[AHMINI ASSURANCE, R.C MEDECIN, AUTOMOBILE, IN..."
1,"[AUTOMOBILE ASSURANCE FRONTIERE TEMPORAIRE, DE...",[AUTOMOBILE]
2,"[SANTE ET PREVOYANCE, TOUS RISQUES MONTAGE, OM...","[R.C MEDECIN, AUTOMOBILE, INCENDIE RISQUES IND..."
3,"[SANTE ET PREVOYANCE, POLICE A ALIMENTER(CORPS...","[AUTOMOBILE AMICALE, INCENDIE RISQUES INDUSTRIEL]"
4,[R.C CHEF DE FAMILLE DETENTEUR D UN ANIMAL DOM...,[]


In [26]:
client_data['final_recommendations'] = client_data.apply(
    lambda row: list(set(row['recommendations']) - set(row['Produit'])),
    axis=1
)
client_data.head()

,ClientID,Nom,Prénom,CIN,Sexe,SituationFamiliale,Profession,RevenusMensuels,Adresse,Email,...,Branche,Produit,DateEffet,DateEcheance,StatutContrat,StatutPaiement,PrimeAnnuelle,ClientScore,recommendations,final_recommendations
0,1,Ben Ali El Amri,Ahmed Salah,82387512,F,Marié,Médecin,4041,"97 Rue Imbert - Mutuelleville, Sfax",ahmed.salah@gmail.com,...,VIE,"[RENTE EDUCATION (AS), CREDITS COMMERCIAUX (AS...","{2023-04-29 00:00:00: 1, 2024-06-19 00:00:00: ...","{2025-05-07 00:00:00: 1, 2025-05-23 00:00:00: ...","{'Actif': 2, 'Suspendu': 1, 'Résilié': 1}","{'Retard': 2, 'À jour': 1, 'Non payé': 1}",13882,0.752034,"[AHMINI ASSURANCE, R.C MEDECIN, AUTOMOBILE, IN...","[INCENDIE RISQUES INDUSTRIEL, SANTE ET PREVOYA..."
1,2,Ben Ali Ben Hassen,Ahmed Salah,11727315,M,Marié,Artisan,1813,"106 Rue Giraud - Centre-Ville, Mahdia",ahmed.salah@gmail.com,...,AUTOMOBILE,"[AUTOMOBILE ASSURANCE FRONTIERE TEMPORAIRE, DE...","{2021-08-20 00:00:00: 1, 2023-04-05 00:00:00: ...","{2026-01-26 00:00:00: 1, 2027-01-17 00:00:00: ...","{'Résilié': 3, 'Actif': 2, 'Suspendu': 1}","{'À jour': 2, 'Retard': 2, 'Non payé': 2}",17198,0.721292,[AUTOMOBILE],[AUTOMOBILE]
2,3,Ben Ali Tlili,Ahmed Salah,82121072,M,Marié,Médecin,2867,"164 Rue Guichard - Lac 2, Gafsa",ahmed.salah@gmail.com,...,GROUPE-MALADIE,"[SANTE ET PREVOYANCE, TOUS RISQUES MONTAGE, OM...","{2021-03-10 00:00:00: 1, 2021-11-30 00:00:00: ...","{2025-08-26 00:00:00: 1, 2027-03-12 00:00:00: ...","{'Actif': 4, 'Résilié': 3, 'Suspendu': 1}","{'Non payé': 4, 'À jour': 2, 'Retard': 2}",15209,0.769237,"[R.C MEDECIN, AUTOMOBILE, INCENDIE RISQUES IND...","[AUTOMOBILE, R.C MEDECIN, INCENDIE RISQUES IND..."
3,4,Ben Ali Belhaj,Ahmed Salah,2858647,M,Marié,Ingénieur,2994,"156 Rue Colas - Centre-Ville, Sfax",ahmed.salah@gmail.com,...,GROUPE-MALADIE,"[SANTE ET PREVOYANCE, POLICE A ALIMENTER(CORPS...","{2022-03-18 00:00:00: 1, 2023-08-02 00:00:00: ...","{2025-04-26 00:00:00: 1, 2025-11-27 00:00:00: ...","{'Actif': 4, 'Suspendu': 3, 'Résilié': 1}","{'À jour': 4, 'Retard': 4}",21366,0.828887,"[AUTOMOBILE AMICALE, INCENDIE RISQUES INDUSTRIEL]","[AUTOMOBILE AMICALE, INCENDIE RISQUES INDUSTRIEL]"
4,5,Ben Ali Mzoughi,Ahmed Salah,52452825,F,Divorcé,Étudiant,5729,"46 Rue Weber - Ezzahra, Kasserine",ahmed.salah@gmail.com,...,RISQUES DIVERS,[R.C CHEF DE FAMILLE DETENTEUR D UN ANIMAL DOM...,"{2020-12-11 00:00:00: 1, 2022-05-22 00:00:00: ...","{2025-07-12 00:00:00: 1, 2027-04-28 00:00:00: ...","{'Résilié': 1, 'Actif': 1, 'Suspendu': 1}",{'À jour': 3},8519,0.708820,[],[]


In [28]:
clients_final = client_data[client_data['final_recommendations'].apply(len)>0]

In [29]:
clients_final.shape

(10981, 24)

In [30]:
clients_final.head()

,ClientID,Nom,Prénom,CIN,Sexe,SituationFamiliale,Profession,RevenusMensuels,Adresse,Email,...,Branche,Produit,DateEffet,DateEcheance,StatutContrat,StatutPaiement,PrimeAnnuelle,ClientScore,recommendations,final_recommendations
0,1,Ben Ali El Amri,Ahmed Salah,82387512,F,Marié,Médecin,4041,"97 Rue Imbert - Mutuelleville, Sfax",ahmed.salah@gmail.com,...,VIE,"[RENTE EDUCATION (AS), CREDITS COMMERCIAUX (AS...","{2023-04-29 00:00:00: 1, 2024-06-19 00:00:00: ...","{2025-05-07 00:00:00: 1, 2025-05-23 00:00:00: ...","{'Actif': 2, 'Suspendu': 1, 'Résilié': 1}","{'Retard': 2, 'À jour': 1, 'Non payé': 1}",13882,0.752034,"[AHMINI ASSURANCE, R.C MEDECIN, AUTOMOBILE, IN...","[INCENDIE RISQUES INDUSTRIEL, SANTE ET PREVOYA..."
1,2,Ben Ali Ben Hassen,Ahmed Salah,11727315,M,Marié,Artisan,1813,"106 Rue Giraud - Centre-Ville, Mahdia",ahmed.salah@gmail.com,...,AUTOMOBILE,"[AUTOMOBILE ASSURANCE FRONTIERE TEMPORAIRE, DE...","{2021-08-20 00:00:00: 1, 2023-04-05 00:00:00: ...","{2026-01-26 00:00:00: 1, 2027-01-17 00:00:00: ...","{'Résilié': 3, 'Actif': 2, 'Suspendu': 1}","{'À jour': 2, 'Retard': 2, 'Non payé': 2}",17198,0.721292,[AUTOMOBILE],[AUTOMOBILE]
2,3,Ben Ali Tlili,Ahmed Salah,82121072,M,Marié,Médecin,2867,"164 Rue Guichard - Lac 2, Gafsa",ahmed.salah@gmail.com,...,GROUPE-MALADIE,"[SANTE ET PREVOYANCE, TOUS RISQUES MONTAGE, OM...","{2021-03-10 00:00:00: 1, 2021-11-30 00:00:00: ...","{2025-08-26 00:00:00: 1, 2027-03-12 00:00:00: ...","{'Actif': 4, 'Résilié': 3, 'Suspendu': 1}","{'Non payé': 4, 'À jour': 2, 'Retard': 2}",15209,0.769237,"[R.C MEDECIN, AUTOMOBILE, INCENDIE RISQUES IND...","[AUTOMOBILE, R.C MEDECIN, INCENDIE RISQUES IND..."
3,4,Ben Ali Belhaj,Ahmed Salah,2858647,M,Marié,Ingénieur,2994,"156 Rue Colas - Centre-Ville, Sfax",ahmed.salah@gmail.com,...,GROUPE-MALADIE,"[SANTE ET PREVOYANCE, POLICE A ALIMENTER(CORPS...","{2022-03-18 00:00:00: 1, 2023-08-02 00:00:00: ...","{2025-04-26 00:00:00: 1, 2025-11-27 00:00:00: ...","{'Actif': 4, 'Suspendu': 3, 'Résilié': 1}","{'À jour': 4, 'Retard': 4}",21366,0.828887,"[AUTOMOBILE AMICALE, INCENDIE RISQUES INDUSTRIEL]","[AUTOMOBILE AMICALE, INCENDIE RISQUES INDUSTRIEL]"
5,6,Ben Ali Cherif,Ahmed Salah,77696424,M,Veuf,Médecin,3713,"157 Rue Bertrand - Mutuelleville, Nabeul",ahmed.salah@gmail.com,...,TRANSPORT,"[POLICE ABONNEMENT(FACULTE MARITIME), POLICE A...","{2024-02-12 00:00:00: 1, 2023-03-13 00:00:00: ...","{2024-12-27 00:00:00: 1, 2025-12-07 00:00:00: ...","{'Résilié': 3, 'Actif': 1}","{'Retard': 2, 'Non payé': 1, 'À jour': 1}",4932,0.486023,"[R.C MEDECIN, AUTOMOBILE, INCENDIE RISQUES IND...","[AUTOMOBILE, R.C MEDECIN, INCENDIE RISQUES IND..."
